In [1]:
import cv2 as cv
import numpy as np

# Load the pre-trained Haar Cascade classifiers for face and eye detection
face_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_eye.xml')

# Open the camera
capture = cv.VideoCapture(0)
while True:
    # Capture frame-by-frame
    ret, frame = capture.read()
    
    # Convert to grayscale for face and eye detection
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    
    # Detect faces in the grayscale frame
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)
    
    for (x, y, w, h) in faces:
        cv.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        # Region of interest for eyes within the detected face
        roi_gray = gray[y:y + h, x:x + w]
        roi_color = frame[y:y + h, x:x + w]
        
        eyes = eye_cascade.detectMultiScale(roi_gray, 1.1, 4)
        index = 0
        eye_1 = [None, None, None, None]
        eye_2 = [None, None, None, None]
        
        for (ex, ey, ew, eh) in eyes:
            if index == 0:
                eye_1 = [ex, ey, ew, eh]
            elif index == 1:
                eye_2 = [ex, ey, ew, eh]
            cv.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 0, 255), 2)
            index += 1
        
        if (eye_1[0] is not None) and (eye_2[0] is not None):
            if eye_1[0] < eye_2[0]:
                left_eye = eye_1
                right_eye = eye_2
            else:
                left_eye = eye_2
                right_eye = eye_1
            
            left_eye_center = (int(left_eye[0] + (left_eye[2] / 2)), int(left_eye[1] + (left_eye[3] / 2)))
            right_eye_center = (int(right_eye[0] + (right_eye[2] / 2)), int(right_eye[1] + (right_eye[3] / 2)))
            
            left_eye_x = left_eye_center[0]
            left_eye_y = left_eye_center[1]
            right_eye_x = right_eye_center[0]
            right_eye_y = right_eye_center[1]
            
            delta_x = right_eye_x - left_eye_x
            delta_y = right_eye_y - left_eye_y
            
            # Slope of line formula
            angle = np.arctan(delta_y / delta_x)
            
            # Converting radians to degrees
            angle = (angle * 180) / np.pi
            
            # Provided a margin of error of 10 degrees
            if angle > 10:
                cv.putText(frame, 'RIGHT TILT :' + str(int(angle)) + ' degrees',
                           (20, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv.LINE_AA)
            elif angle < -10:
                cv.putText(frame, 'LEFT TILT :' + str(int(angle)) + ' degrees',
                           (20, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv.LINE_AA)
            else:
                cv.putText(frame, 'STRAIGHT :', (20, 30),
                           cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv.LINE_AA)
    
    # Display the resulting frame
    cv.imshow('Frame', frame)
    
    # Press 'a' to exit the video display window
    if cv.waitKey(1) & 0xFF == ord('a'):
        break

# When everything done, release the capture
capture.release()
cv.destroyAllWindows()